In [2]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

# install findspark using pip
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 199 kB 47.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=accabf2822718a8f125d96391066f8aed5b580cc7f2b8478a29e503d9da6be9c
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
import findspark 
findspark.init()
from pyspark.sql import SparkSession

In [4]:
from pyspark.sql import Row

In [11]:
#buat session
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [15]:
def mapper(line):
  fields = line.split(",")
  return Row(ID=int(fields[0]), name=str(fields[1].encode("utf-8")), age=int(fields[2]), numFriends=int(fields[3]))

In [19]:
lines = spark.sparkContext.textFile("/content/fakefriends.csv") 

In [ ]:
lines 

In [21]:
people = lines.map(mapper)

In [ ]:
people 

In [23]:
schemaPeople = spark.createDataFrame(people).cache() 
schemaPeople.createOrReplaceTempView("people") 

In [24]:
schemaPeople 

DataFrame[ID: bigint, name: string, age: bigint, numFriends: bigint]

In [25]:
teenagers = spark.sql("SELECT * FROM people WHERE age >=13 AND age <=19")

In [29]:
for teen in teenagers.collect():
  print(teen)

Row(ID=21, name="b'Miles'", age=19, numFriends=268)
Row(ID=52, name="b'Beverly'", age=19, numFriends=269)
Row(ID=54, name="b'Brunt'", age=19, numFriends=5)
Row(ID=106, name="b'Beverly'", age=18, numFriends=499)
Row(ID=115, name="b'Dukat'", age=18, numFriends=397)
Row(ID=133, name="b'Quark'", age=19, numFriends=265)
Row(ID=136, name="b'Will'", age=19, numFriends=335)
Row(ID=225, name="b'Elim'", age=19, numFriends=106)
Row(ID=304, name="b'Will'", age=19, numFriends=404)
Row(ID=341, name="b'Data'", age=18, numFriends=326)
Row(ID=366, name="b'Keiko'", age=19, numFriends=119)
Row(ID=373, name="b'Quark'", age=19, numFriends=272)
Row(ID=377, name="b'Beverly'", age=18, numFriends=418)
Row(ID=404, name="b'Kasidy'", age=18, numFriends=24)
Row(ID=409, name="b'Nog'", age=19, numFriends=267)
Row(ID=439, name="b'Data'", age=18, numFriends=417)
Row(ID=444, name="b'Keiko'", age=18, numFriends=472)
Row(ID=492, name="b'Dukat'", age=19, numFriends=36)
Row(ID=494, name="b'Kasidy'", age=18, numFriends=194)

In [28]:
teenagers.show()

+---+----------+---+----------+
| ID|      name|age|numFriends|
+---+----------+---+----------+
| 21|  b'Miles'| 19|       268|
| 52|b'Beverly'| 19|       269|
| 54|  b'Brunt'| 19|         5|
|106|b'Beverly'| 18|       499|
|115|  b'Dukat'| 18|       397|
|133|  b'Quark'| 19|       265|
|136|   b'Will'| 19|       335|
|225|   b'Elim'| 19|       106|
|304|   b'Will'| 19|       404|
|341|   b'Data'| 18|       326|
|366|  b'Keiko'| 19|       119|
|373|  b'Quark'| 19|       272|
|377|b'Beverly'| 18|       418|
|404| b'Kasidy'| 18|        24|
|409|    b'Nog'| 19|       267|
|439|   b'Data'| 18|       417|
|444|  b'Keiko'| 18|       472|
|492|  b'Dukat'| 19|        36|
|494| b'Kasidy'| 18|       194|
+---+----------+---+----------+



In [30]:
schemaPeople.groupBy("age").count().orderBy("age").show(5)

+---+-----+
|age|count|
+---+-----+
| 18|    8|
| 19|   11|
| 20|    5|
| 21|    8|
| 22|    7|
+---+-----+
only showing top 5 rows



In [32]:
spark.stop() 

In [43]:
!spark-submit spark-sql.py > spark-sql.txt

22/07/05 03:26:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/07/05 03:26:53 INFO SparkContext: Running Spark version 3.0.0
22/07/05 03:26:53 INFO ResourceUtils: ==============================================================
22/07/05 03:26:53 INFO ResourceUtils: Resources for spark.driver:

22/07/05 03:26:53 INFO ResourceUtils: ==============================================================
22/07/05 03:26:53 INFO SparkContext: Submitted application: SparkSQL
22/07/05 03:26:53 INFO SecurityManager: Changing view acls to: root
22/07/05 03:26:53 INFO SecurityManager: Changing modify acls to: root
22/07/05 03:26:53 INFO SecurityManager: Changing view acls groups to: 
22/07/05 03:26:53 INFO SecurityManager: Changing modify acls groups to: 
22/07/05 03:26:53 INFO SecurityManager: SecurityManager: authentication disabled; ui acls di

### USING DATAFRAME INSTEAD OF RDD

In [5]:
from pyspark.sql import functions as func

In [6]:
#buat session
spark = SparkSession.builder.appName("FriendsByAge").getOrCreate()

In [7]:
lines = spark.read.option("header","true").option("inferSchema","true").csv("fakefriends-header.csv")


In [8]:
lines

DataFrame[ID: int, Name: string, Age: int, NumFriends: int]

In [9]:
friendsByAge = lines.select("Age","NumFriends")

In [10]:
friendsByAge

DataFrame[Age: int, NumFriends: int]

In [11]:
friendsByAge.show()

+---+----------+
|Age|NumFriends|
+---+----------+
| 33|       385|
| 26|         2|
| 55|       221|
| 40|       465|
| 68|        21|
| 59|       318|
| 37|       220|
| 54|       307|
| 38|       380|
| 27|       181|
| 53|       191|
| 57|       372|
| 54|       253|
| 56|       444|
| 43|        49|
| 36|        49|
| 22|       323|
| 35|        13|
| 45|       455|
| 60|       246|
+---+----------+
only showing top 20 rows



In [12]:
friendsByAge.groupBy("Age").avg("NumFriends").show()

+---+------------------+
|Age|   avg(NumFriends)|
+---+------------------+
| 31|            267.25|
| 65|             298.2|
| 53|222.85714285714286|
| 34|             245.5|
| 28|             209.1|
| 26|242.05882352941177|
| 27|           228.125|
| 44| 282.1666666666667|
| 22|206.42857142857142|
| 47|233.22222222222223|
| 52| 340.6363636363636|
| 40| 250.8235294117647|
| 20|             165.0|
| 57| 258.8333333333333|
| 54| 278.0769230769231|
| 48|             281.4|
| 19|213.27272727272728|
| 64| 281.3333333333333|
| 41|268.55555555555554|
| 43|230.57142857142858|
+---+------------------+
only showing top 20 rows



In [13]:
friendsByAge.groupBy("Age").avg("NumFriends").sort("Age").show()

+---+------------------+
|Age|   avg(NumFriends)|
+---+------------------+
| 18|           343.375|
| 19|213.27272727272728|
| 20|             165.0|
| 21|           350.875|
| 22|206.42857142857142|
| 23|             246.3|
| 24|             233.8|
| 25|197.45454545454547|
| 26|242.05882352941177|
| 27|           228.125|
| 28|             209.1|
| 29|215.91666666666666|
| 30| 235.8181818181818|
| 31|            267.25|
| 32| 207.9090909090909|
| 33| 325.3333333333333|
| 34|             245.5|
| 35|           211.625|
| 36|             246.6|
| 37|249.33333333333334|
+---+------------------+
only showing top 20 rows



In [14]:
#sum
friendsByAge.groupBy("Age").sum("NumFriends").sort("Age").show()

+---+---------------+
|Age|sum(NumFriends)|
+---+---------------+
| 18|           2747|
| 19|           2346|
| 20|            825|
| 21|           2807|
| 22|           1445|
| 23|           2463|
| 24|           1169|
| 25|           2172|
| 26|           4115|
| 27|           1825|
| 28|           2091|
| 29|           2591|
| 30|           2594|
| 31|           2138|
| 32|           2287|
| 33|           3904|
| 34|           1473|
| 35|           1693|
| 36|           2466|
| 37|           2244|
+---+---------------+
only showing top 20 rows



In [15]:
#formated more nicely
friendsByAge.groupBy("Age").agg(func.round(func.avg("NumFriends"),2)).sort("Age").show()

+---+-------------------------+
|Age|round(avg(NumFriends), 2)|
+---+-------------------------+
| 18|                   343.38|
| 19|                   213.27|
| 20|                    165.0|
| 21|                   350.88|
| 22|                   206.43|
| 23|                    246.3|
| 24|                    233.8|
| 25|                   197.45|
| 26|                   242.06|
| 27|                   228.13|
| 28|                    209.1|
| 29|                   215.92|
| 30|                   235.82|
| 31|                   267.25|
| 32|                   207.91|
| 33|                   325.33|
| 34|                    245.5|
| 35|                   211.63|
| 36|                    246.6|
| 37|                   249.33|
+---+-------------------------+
only showing top 20 rows



In [16]:
#formated more nicely
friendsByAge.groupBy("Age").agg(func.round(func.avg("NumFriends"),2).alias("friends_avg")).sort("Age").show()

+---+-----------+
|Age|friends_avg|
+---+-----------+
| 18|     343.38|
| 19|     213.27|
| 20|      165.0|
| 21|     350.88|
| 22|     206.43|
| 23|      246.3|
| 24|      233.8|
| 25|     197.45|
| 26|     242.06|
| 27|     228.13|
| 28|      209.1|
| 29|     215.92|
| 30|     235.82|
| 31|     267.25|
| 32|     207.91|
| 33|     325.33|
| 34|      245.5|
| 35|     211.63|
| 36|      246.6|
| 37|     249.33|
+---+-----------+
only showing top 20 rows



In [17]:
spark.stop()

### **CUSTOM SCHEMA**

In [19]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

spark = SparkSession.builder.appName("MinTemperatures").getOrCreate()

schema = StructType([ \
                     StructField("stationID", StringType(), True), \
                     StructField("date", IntegerType(), True), \
                     StructField("measure_type", StringType(), True), \
                     StructField("temperature", FloatType(), True)]) 

df = spark.read.schema(schema).csv("1800.csv")
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: float (nullable = true)



In [20]:
#filter out all but tmin entries
minTemps = df.filter(df.measure_type == "TMIN")

#select only stationid and temperature
stationTemps = minTemps.select("stationID","temperature")

#aggregate to find minimun temperature for every station
minTempsByStation = stationTemps.groupBy("stationID").min("temperature")
minTempsByStation.show()

+-----------+----------------+
|  stationID|min(temperature)|
+-----------+----------------+
|ITE00100554|          -148.0|
|EZE00100082|          -135.0|
+-----------+----------------+



In [21]:
#convert temperature to fahrenheit and sort dataset

minTempsByStationF = minTempsByStation.withColumn("temperature",
                                                  func.round(func.col("min(temperature)") *0.1 *(9.0 / 5.0)+32.0, 2))\
                                                  .select("stationID", "temperature").sort("temperature")

In [23]:
#collct, format and print the result

results = minTempsByStationF.collect()

for result in results:
  print(result[0]+"\t{:.2f}F".format(result[1]))

ITE00100554	5.36F
EZE00100082	7.70F


In [24]:
spark.stop()